pca_bert_150features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Bert/bert_withstopwordchangelevels_150_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,...,0.126189,-0.014984,-0.143537,-0.027213,-0.204920,-0.023267,-0.175738,0.693520,-0.057460,0.055772
1,1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,...,-0.264938,-0.593206,-0.243180,-0.070606,-0.127084,-0.094425,-0.495715,-0.472812,0.428107,0.269283
2,2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,...,-0.010397,0.684243,0.105883,0.006904,0.026404,-0.166052,0.484260,0.268894,-0.335252,-0.436707
3,3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,...,-0.769572,0.509927,0.897845,0.622906,-0.366820,-0.410176,-0.051106,-0.095815,-0.210337,0.326416
4,4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,...,0.265948,0.116375,-0.214717,-0.069630,-0.329385,-0.414796,0.262825,0.510518,0.089978,0.098790


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-15.959655,7.610355,-4.614255,3.683309,2.789502,-1.250055,-1.805187,-10.514873,-4.086176,0.203140,...,0.126189,-0.014984,-0.143537,-0.027213,-0.204920,-0.023267,-0.175738,0.693520,-0.057460,0.055772
1,-13.495674,4.785825,-0.513378,4.535705,-0.666486,-2.018108,-2.203631,5.270856,-0.704172,1.341339,...,-0.264938,-0.593206,-0.243180,-0.070606,-0.127084,-0.094425,-0.495715,-0.472812,0.428107,0.269283
2,5.556126,-6.071359,15.245860,-6.528944,1.345797,5.089355,-4.129077,-5.236971,-3.814316,-2.739572,...,-0.010397,0.684243,0.105883,0.006904,0.026404,-0.166052,0.484260,0.268894,-0.335252,-0.436707
3,-4.445179,1.293825,-4.332579,-6.459947,3.055774,8.987284,7.318152,-5.308832,-3.731958,-2.567814,...,-0.769572,0.509927,0.897845,0.622906,-0.366820,-0.410176,-0.051106,-0.095815,-0.210337,0.326416
4,-12.149864,1.734085,-7.929423,-3.487395,3.067365,4.222421,-4.707823,2.094515,0.077916,1.247958,...,0.265948,0.116375,-0.214717,-0.069630,-0.329385,-0.414796,0.262825,0.510518,0.089978,0.098790


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 150), (524, 150))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3986234035178381


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472131,0.472131,0.620087,0.449896,0.29771,0.29771,0.365022,0.276115


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2545    0.0892    0.1321       157
           2     0.4507    0.8481    0.5886       237
           3     0.2609    0.0462    0.0784       130

    accuracy                         0.4218       524
   macro avg     0.3220    0.3278    0.2664       524
weighted avg     0.3448    0.4218    0.3252       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472131,0.472131,0.620087,0.449896,0.297710,0.297710,0.365022,0.276115
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.421756,0.421756,0.344821,0.325239


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5903    0.4709    0.5239       361
           2     0.5988    0.7505    0.6661       553
           3     0.5649    0.4412    0.4954       306

    accuracy                         0.5902      1220
   macro avg     0.5847    0.5542    0.5618      1220
weighted avg     0.5878    0.5902    0.5812      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3190    0.2357    0.2711       157
           2     0.4674    0.5738    0.5152       237
           3     0.2393    0.2154    0.2267       130

    accuracy                         0.3836       524
   macro avg     0.3419    0.3416    0.3376       524
weighted avg     0.3663    0.3836    0.3705       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.625171,0.442521,0.437023,0.437023,0.370417,0.357188
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.433206,0.433206,0.372238,0.345913
2,Adaboost,0.590164,0.590164,0.587784,0.581220,0.383588,0.383588,0.366320,0.370461


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9916    0.9861    0.9889       361
           2     0.9668    1.0000    0.9831       553
           3     1.0000    0.9444    0.9714       306

    accuracy                         0.9820      1220
   macro avg     0.9861    0.9769    0.9811      1220
weighted avg     0.9825    0.9820    0.9819      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3243    0.2293    0.2687       157
           2     0.4678    0.6751    0.5527       237
           3     0.2254    0.1231    0.1592       130

    accuracy                         0.4046       524
   macro avg     0.3392    0.3425    0.3268       524
weighted avg     0.3647    0.4046    0.3700       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.625171,0.442521,0.437023,0.437023,0.370417,0.357188
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.433206,0.433206,0.372238,0.345913
2,Adaboost,0.590164,0.590164,0.587784,0.581220,0.383588,0.383588,0.366320,0.370461
3,GBM,0.981967,0.981967,0.982471,0.981891,0.404580,0.404580,0.364679,0.369962


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4812    0.5679    0.5210       361
           2     0.6787    0.5118    0.5835       553
           3     0.4775    0.5882    0.5271       306

    accuracy                         0.5475      1220
   macro avg     0.5458    0.5560    0.5439      1220
weighted avg     0.5698    0.5475    0.5508      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2823    0.3758    0.3224       157
           2     0.3869    0.2236    0.2834       237
           3     0.2360    0.3231    0.2727       130

    accuracy                         0.2939       524
   macro avg     0.3017    0.3075    0.2929       524
weighted avg     0.3181    0.2939    0.2924       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472131,0.472131,0.620087,0.449896,0.297710,0.297710,0.365022,0.276115
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.421756,0.421756,0.344821,0.325239
2,SVM,0.547541,0.547541,0.569770,0.550849,0.293893,0.293893,0.318093,0.292449


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5066    0.5291    0.5176       361
           2     0.5761    0.7324    0.6449       553
           3     0.4857    0.2222    0.3049       306

    accuracy                         0.5443      1220
   macro avg     0.5228    0.4946    0.4892      1220
weighted avg     0.5329    0.5443    0.5220      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2353    0.2548    0.2446       157
           2     0.4331    0.5190    0.4722       237
           3     0.1857    0.1000    0.1300       130

    accuracy                         0.3359       524
   macro avg     0.2847    0.2913    0.2823       524
weighted avg     0.3125    0.3359    0.3191       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.625171,0.442521,0.437023,0.437023,0.370417,0.357188
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.433206,0.433206,0.372238,0.345913
2,Adaboost,0.590164,0.590164,0.587784,0.581220,0.383588,0.383588,0.366320,0.370461
3,GBM,0.981967,0.981967,0.982471,0.981891,0.404580,0.404580,0.364679,0.369962
4,SVM,0.575410,0.575410,0.572116,0.560929,0.337786,0.337786,0.317506,0.323968
5,KNN,0.544262,0.544262,0.532875,0.521968,0.335878,0.335878,0.312459,0.319110


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4010    0.6454    0.4947       361
           2     0.6159    0.5190    0.5633       553
           3     0.3988    0.2255    0.2881       306

    accuracy                         0.4828      1220
   macro avg     0.4719    0.4633    0.4487      1220
weighted avg     0.4979    0.4828    0.4740      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3308    0.5541    0.4143       157
           2     0.5460    0.4008    0.4623       237
           3     0.2644    0.1769    0.2120       130

    accuracy                         0.3912       524
   macro avg     0.3804    0.3773    0.3629       524
weighted avg     0.4116    0.3912    0.3858       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.512295,0.512295,0.625171,0.442521,0.437023,0.437023,0.370417,0.357188
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.433206,0.433206,0.372238,0.345913
2,Adaboost,0.590164,0.590164,0.587784,0.581220,0.383588,0.383588,0.366320,0.370461
3,GBM,0.981967,0.981967,0.982471,0.981891,0.404580,0.404580,0.364679,0.369962
4,SVM,0.575410,0.575410,0.572116,0.560929,0.337786,0.337786,0.317506,0.323968
5,KNN,0.544262,0.544262,0.532875,0.521968,0.335878,0.335878,0.312459,0.319110
6,GNB,0.482787,0.482787,0.497869,0.473972,0.391221,0.391221,0.411641,0.385806
